<a href="https://colab.research.google.com/github/sgywhetstone/WiDS-Datathon-2020/blob/master/WiDS_2020_ESL_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import files 
uploaded = files.upload() 

Saving samplesubmission.csv to samplesubmission.csv
Saving solution_template.csv to solution_template.csv
Saving training_v2.csv to training_v2.csv
Saving unlabeled.csv to unlabeled.csv
Saving WiDS Datathon 2020 Dictionary.csv to WiDS Datathon 2020 Dictionary.csv


In [0]:
#Read data into DataFrames
train = pd.read_csv('training_v2.csv')
test = pd.read_csv('unlabeled.csv')
st = pd.read_csv('solution_template.csv')
ss = pd.read_csv('samplesubmission.csv')
dictionary = pd.read_csv('WiDS Datathon 2020 Dictionary.csv')

In [0]:
# Dropping patient_id for now
train = train.copy().drop('patient_id', axis = 1)
test = test.copy().drop('patient_id', axis = 1)

# Dropping readmission_status for now
train = train.copy().drop('readmission_status', axis = 1)
test = test.copy().drop('readmission_status', axis = 1)

In [48]:
#Make Copies for first model
naive_train = train.copy()
naive_test = test.copy()

#Delete columns with too many missing entries
# Drop columns based on threshold limit
threshold = len(naive_train) * 0.90
naive_train_thresh=naive_train.dropna(axis=1, thresh=threshold)
# View columns in the dataset
naive_train_thresh.shape

#Delete all rows with missing data
print("Orginal shape before dropna()" ,naive_train_thresh.shape)
naive_drop = naive_train_thresh.dropna()
print("Shape after dropna()",naive_drop.shape)

Orginal shape before dropna() (91713, 81)
Shape after dropna() (59617, 81)


In [49]:
#Check for no NaNs
naive_drop.isna().sum().sum()

0

In [50]:
#Edit test data to correct number of columns
naive_test = test[naive_drop.columns]



#Concat test and training data
dataset = pd.concat(objs=[naive_drop, naive_test], axis=0)

naive_test.shape

(39308, 81)

In [51]:
naive_test.shape

(39308, 81)

In [0]:
#Identify Categorical variables
#For Train data
l1 = []
col = dataset.columns
for i in col:
    if dataset[i].nunique() <= 15:
        l1.append(i)

#Encode Categorical variables
dataset[l1] = dataset[l1].astype('category')
for col in l1:
   dataset[col] = dataset[col].cat.codes

In [0]:
train_split = dataset.iloc[:59617]
test_split = dataset.iloc[59617:]

In [54]:
test_split.shape

(39308, 81)

In [0]:
#Split data into training and validation

from sklearn.model_selection import train_test_split

Train, Validation = train_test_split(train_split, test_size = 0.3, random_state = 888)

x_train = Train.copy().drop('hospital_death', axis = 1)
y_train = Train[['hospital_death']]
x_val = Validation.copy().drop('hospital_death', axis = 1)
y_val = Validation[['hospital_death']]

x_test = test_split.copy().drop('hospital_death', axis = 1)
y_test = test_split[['hospital_death']]

In [56]:
import keras
keras.__version__

'2.2.5'

In [0]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(80,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [58]:
history = model.fit(x_train,
                    y_train,
                   epochs=100,
                   batch_size=512,
                   validation_data=(x_val, y_val))

Train on 41731 samples, validate on 17886 samples
Epoch 1/100
41731/41731 [==============================] - 1s 16us/step - loss: 3.3011 - acc: 0.7926 - val_loss: 1.3772 - val_acc: 0.9145
Epoch 2/100
41731/41731 [==============================] - 0s 10us/step - loss: 1.3795 - acc: 0.9144 - val_loss: 1.3752 - val_acc: 0.9147
Epoch 3/100
41731/41731 [==============================] - 0s 10us/step - loss: 1.3789 - acc: 0.9145 - val_loss: 1.3752 - val_acc: 0.9147
Epoch 4/100
41731/41731 [==============================] - 0s 9us/step - loss: 1.3789 - acc: 0.9145 - val_loss: 1.3752 - val_acc: 0.9147
Epoch 5/100
41731/41731 [==============================] - 0s 9us/step - loss: 1.3789 - acc: 0.9145 - val_loss: 1.3752 - val_acc: 0.9147
Epoch 6/100
41731/41731 [==============================] - 0s 10us/step - loss: 1.3789 - acc: 0.9145 - val_loss: 1.3752 - val_acc: 0.9147
Epoch 7/100
41731/41731 [==============================] - 0s 9us/step - loss: 1.3789 - acc: 0.9145 - val_loss: 1.3752 - val

In [0]:
output = model.predict(x_test)

In [67]:
output

array([[0.       ],
       [0.4792382],
       [0.4792382],
       ...,
       [0.       ],
       [0.       ],
       [0.4792382]], dtype=float32)

In [74]:
test["hospital_death"] = output
test[["encounter_id","hospital_death"]].to_csv("submission_1.csv",index=False)

test.head()

,encounter_id,hospital_id,hospital_death,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,...,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,2,7,0.000000,56.0,21.102439,0,Caucasian,M,180.3,Emergency Department,Accident & Emergency,1105,admit,Neuro ICU,0.104167,68.6,3.7,301.0,405.01,0,0.0,NaN,19.0,0.67,NaN,2.0,5.0,0.0,1.0,276.0,105.0,42.0,0.0,57.0,NaN,NaN,NaN,NaN,5.0,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.13,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
1,5,178,0.479238,NaN,19.500595,1,Caucasian,F,145.0,Operating Room,Operating Room / Recovery,1043,admit,Med-Surg ICU,0.230556,41.0,NaN,217.0,1502.02,1,0.0,NaN,NaN,NaN,NaN,4.0,6.0,0.0,4.0,NaN,60.0,NaN,0.0,53.0,NaN,NaN,NaN,NaN,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic
2,7,86,0.479238,66.0,32.518596,1,African American,M,170.2,Operating Room,Operating Room / Recovery,1035,admit,CCU-CTICU,0.140972,94.2,NaN,302.0,1208.06,1,0.0,NaN,22.0,1.84,NaN,2.0,5.0,0.0,1.0,81.0,124.0,39.2,0.0,115.0,NaN,NaN,NaN,NaN,31.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.03,0.02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular
3,8,163,0.000000,64.0,20.798520,0,Caucasian,M,182.8,Operating Room,Operating Room / Recovery,1078,admit,Med-Surg ICU,0.002083,69.5,1.9,303.0,211.03,0,0.0,0.8,24.0,0.75,0.7,4.0,6.0,0.0,1.0,122.0,111.0,32.0,1.0,48.0,36.0,36.0,142.0,7.35,30.0,133.0,...,1.4,1.4,NaN,NaN,248.0,248.0,4.0,4.0,133.0,133.0,9.7,9.7,36.0,36.0,7.37,7.35,142.0,57.0,202.857143,114.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.19,0.17,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Respiratory,Respiratory
4,10,59,0.479238,56.0,NaN,0,Caucasian,M,NaN,Emergency Department,Accident & Emergency,1012,admit,Neuro ICU,0.004861,66.4,NaN,217.0,1501.01,1,0.0,NaN,11.0,0.52,NaN,1.0,4.0,0.0,1.0,192.0,94.0,32.8,0.0,41.0,NaN,NaN,NaN,NaN,5.0,134.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic


In [77]:
test[["encounter_id","hospital_death"]].head()

,encounter_id,hospital_death
0,2,0.000000
1,5,0.479238
2,7,0.479238
3,8,0.000000
4,10,0.479238
